## 環境初始化

In [ ]:
# pytorch 套件
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torchvision
from torchvision import datasets, transforms

# 其他套件載入
import random
import numpy as np
import matplotlib.pyplot as plt
import cv2 # opencv-python
import os

# scikit-learn
from sklearn.datasets import load_iris # 載入鳶尾花資料集
from sklearn.model_selection import train_test_split # 切割訓練、測試資料

## 1. Pytorch

In [ ]:
# 創建兩個純量
scalar_1 = torch.tensor([5.0])
scalar_2 = torch.tensor([3.0])

In [ ]:
# 加法
result = scalar_1 + scalar_2
print(f'加法計算結果：{result}')

In [ ]:
# 減法
result = scalar_1 - scalar_2
print(f'減法計算結果：{result}')

In [ ]:
# 乘法
result = scalar_1 * scalar_2
print(f'乘法計算結果：{result}')

In [ ]:
# 除法
result = scalar_1 / scalar_2
print(f'乘法計算結果：{result}')

In [ ]:
# 建立矩陣
# 值從一個均勻分布採樣 U(−k,k), k = 1/sqrt(in_features).
linear_layer = nn.Linear(1, 1)

print(f'隱藏層的權重：{linear_layer.weight}')
print(f'隱藏層的偏差值：{linear_layer.bias}')

In [ ]:
# 輸入值為 5
data = torch.tensor([5], dtype=torch.float32)
data = linear_layer(data)
print(f'輸出值：{data}')

In [ ]:
# 可以控制參數
linear_layer = nn.Linear(1, 1)
linear_layer.weight = nn.Parameter(torch.tensor([[1.0]]))
linear_layer.bias = nn.Parameter(torch.tensor([[1.0]]))
print(f'隱藏層的權重：{linear_layer.weight}')
print(f'隱藏層的偏差值：{linear_layer.bias}')

In [ ]:
# 輸入值為 5
data = torch.tensor([[5]], dtype=torch.float32)
data = linear_layer(data)
print(f'輸出值：{data}')

## 2. Pytorch vs Numpy

In [ ]:
# numpy 權重
weight = np.random.randn(2, 3)
bias = np.zeros(shape=(1, 3))
print(f'Numpy 矩陣 weight 大小：{weight.shape}')
print(f'Numpy 矩陣 bias 大小：{bias.shape}')

Applies a linear transformation to the incoming data: 𝑦 = 𝑥𝐴^𝑇 + 𝑏 -> [Pyotch Linear function](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)


In [ ]:
# torch 權重
linear_layer = nn.Linear(2, 3)
print(f'Pytorch 矩陣 weight 大小：{linear_layer.weight.T.shape}')
print(f'Pytorch 矩陣 bias 大小：{linear_layer.bias.shape}')

In [ ]:
# 自動求導
x = torch.tensor(3.0, requires_grad=True) # 設置需要梯度
y = x**2

# 計算梯度
y.backward() # 自動求導
print(f"x 的值為 {x}，梯度為 {x.grad}")

In [ ]:
# 查看有無GPU，無則指定CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'現在有的計算裝置為：{device}')

In [ ]:
#@title 2.1 Numpy 做矩陣計算

# 用 numpy 產生隨機資料
numpy_data = np.random.randn(10, 4) # (10 , 4) (資料數量, 特徵)

# 建立參數
w1 = np.random.randn(4, 3)          # (4  , 3) (特徵   , 種類)
b1 = np.zeros(shape=(1, 3))

# 矩陣乘法
numpy_data = np.dot(numpy_data, w1) + b1 # (10, 4) dot (4, 3)
print(f'輸出維度：{numpy_data.shape}')

In [ ]:
#@title 2.2 Pytorch 做矩陣計算

# 用 torch 產生隨機資料
tensor_data = torch.randn(10, 4)   # (10 , 4) (資料數量, 特徵)

# 建立參數
linear_layer = nn.Linear(4, 3)     # (4  , 3) (特徵   , 種類)

# 矩陣乘法
tensor_data = linear_layer(tensor_data) # (10, 4) dot (4, 3)
print(f'輸出維度：{tensor_data.shape}')

In [ ]:
#@title 2.3 激活函數
tensor_data = torch.randn(5, 1)  # 10 筆資料, 4 個特徵
print(f'輸出前：{tensor_data}')

activate_layer = nn.Sigmoid() # ReLU, Sigmoid, Tanh

tensor_data = activate_layer(tensor_data)
print(f'輸出後：{tensor_data}')

## 3. 建立分類器

In [ ]:
# 神經網路物件化
# 1. __init__: 模型架構
# 2. forward: 矩陣乘法
class TestNet(nn.Module):

    def __init__(self):
        super(TestNet, self).__init__()

        self.hidden_layer = nn.Linear(4, 3)

    def forward(self, x):         # Input: (10,4)

        x = self.hidden_layer(x)  # Layer: (4, 3)

        return x                  # Output:(10, 3)

test_model = TestNet()

tensor_data = torch.randn(10, 4)  # 10 筆資料, 4 個特徵
tensor_data = test_model(tensor_data)
print(tensor_data.shape)

## 4. 鳶尾花分類 (Pytorch版本)

### 4.1 資料載入以及資料轉換

In [ ]:
# 資料前處理
iris = load_iris()

X = iris.data
y = iris.target

In [ ]:
# 模型切分 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 資料轉換：numpy -> torch
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

In [ ]:
# 使用 Pytorch Dataset
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

# 使用 DataLoader，具備迭代器(Iterator)性質
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

### 4.2 搭建網路、損失函數與優化器

In [ ]:
# 建立網路
class IrisNet(nn.Module):
    def __init__(self):
        super(IrisNet, self).__init__()
        self.hidden_layer = nn.Linear(4, 3)

    def forward(self, x):         # A = (資料數量, 4)
        x = self.hidden_layer(x)  # B = (4      , 3)
        return x                  # A dot B = (資料數量, 3) = C

# 創建模型實例
model = IrisNet()

In [ ]:
# 設置優化器和損失函數
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = nn.CrossEntropyLoss()

### 4.3 建立評估方式與訓練

In [ ]:
# 定義評估方法
def eval_model(model, test_loader):

  model.eval()

  with torch.no_grad():

      correct = 0
      total = 0
      for X_batch, y_batch in test_loader:

          outputs = model(X_batch)
          _, predicted = torch.max(outputs.data, 1)

          total += y_batch.size(0)
          correct += (predicted == y_batch).sum().item()

      print(f'Test Accuracy: {100 * correct / total:.2f}')

In [ ]:
# 評估模型: Before
eval_model(model, test_loader)

In [ ]:
# 監控訓練過程
train_losses = []

num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_train_loss = 0.0
    correct_train = 0
    total_train = 0

    for X_batch, y_batch in train_loader:

        optimizer.zero_grad()

        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        running_train_loss += loss.item()

    train_loss = running_train_loss / len(train_loader)
    train_losses.append(train_loss)

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

In [ ]:
#@title 4.4  視覺化訓練過程
plt.figure(figsize=(12, 5))
plt.plot(train_losses, label='Train Loss', color='blue')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss Over Epochs')
plt.legend()

plt.show()

In [ ]:
# 評估模型: After
eval_model(model, test_loader)

In [ ]:
# 保存模型
torch.save(model.state_dict(), 'model.pth')

## 5. 手寫數字辨識 資料集

In [ ]:
# 加載MNIST數據集
transform = transforms.Compose([transforms.ToTensor()])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)

test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

In [ ]:
print(f'訓練資料集數量：{len(train_dataset)}')
print(f'測試資料集數量：{len(test_dataset)}')

In [ ]:
# 隨機挑出五筆資料
indices = random.sample(range(len(train_dataset)), 5)
samples = [train_dataset[i] for i in indices]

# 顯示圖片
fig, axes = plt.subplots(1, 5, figsize=(15, 3))
for i, (image, label) in enumerate(samples):
    axes[i].imshow(image.squeeze(), cmap='gray')
    axes[i].set_title(f'Label: {label}')
    axes[i].axis('off')

plt.show()

## 6. 圖片資料的前處理

In [ ]:
# 資料前處理
transform = transforms.ToTensor() # numpy -> torch

In [ ]:
# 建立 numpy 資料
data = np.array([[10,9,7,4,1]])

# 轉換資料類別：numpy -> torch
transform(data)

In [ ]:
# 第 3 筆資料
image, label = test_dataset[3]

In [ ]:
# 查看資料
print(f'影像大小：{image.shape}')
print(f'標籤：{label}')

In [ ]:
# 影像呈現
plt.imshow(image[0], cmap='gray')

In [ ]:
# 影像呈現
plt.imshow(image[0], cmap='inferno')
# viridis 藍色到綠色
# plasma  紫色到黃色
# inferno  黑色到黃色

In [ ]:
# 圖片旋轉
image, label = test_dataset[500]

transform = transforms.RandomRotation(degrees=90)
transform_image = transform(image)

plt.imshow(transform_image[0], cmap='gray')

In [ ]:
# 圖片縮放
transform = transforms.Resize((28,42))
transform_image = transform(image)

plt.imshow(transform_image[0], cmap='gray')

In [ ]:
# 圖片剪裁
transform = transforms.CenterCrop((14,14))
transform_image = transform(image)

plt.imshow(transform_image[0], cmap='gray')

In [ ]:
# 水平翻轉
transform = transforms.RandomHorizontalFlip(p=1)
transform_image = transform(image)

plt.imshow(transform_image[0], cmap='gray')

In [ ]:
# 多種前處理
transform = transforms.Compose([      # 包裝多層邏輯

    transforms.CenterCrop((14,14)),   # 裁減
    transforms.RandomHorizontalFlip(p=1)   # 水平翻轉

    ])

transform_image = transform(image)

plt.imshow(transform_image[0], cmap='gray')

## 隨堂練習 A

In [ ]:
#@title 練習 1：設計圖片的前處理流程
### 情境：公司的資料科學家經過統計發現，圖片旋轉45%，接著縮放到原本圖片尺寸的2倍，
### 模型表現最佳，身為資料工程師的你，請實現他的需求，給他一個建立好的 transform 物件
### 提示：需要注意順序，旋轉45 -> 放大2倍 (原始尺寸 28x28)
### 參考：（圖片資料的前處理）：（圖片旋轉）、（圖片縮放）


image, label = test_dataset[1000]
print(f'圖片大小：{image.shape}')

### 實作練習 ###

transform = transforms.Compose([
    ?,
    ?
])

### 實作練習 ###

transform_image = transform(image)
plt.imshow(transform_image[0], cmap='gray')

### 實作結果
# 視覺化的數字有旋轉，且顯示較為模糊

In [ ]:
#@title 練習 2：搭建簡易商品分類模型
### 情境：老闆想使用類神經網路來做商品分類，請幫他實作出來
### 模型設計為兩層：(100, 256) -> (256, 10)
### 參考：4.1 搭建網路）

class Three_layer_model(nn.Module):
    def __init__(self):
        super(Three_layer_model, self).__init__()

        ### 實作練習 ###

        self.layer_1 = ?
        self.layer_2 = ?

        ### 實作練習 ###

    def forward(self, x):

        ### 實作練習 ###

        x = ?
        x = ?

        ### 實作練習 ###

        return x

model = Three_layer_model()

tensor_data = torch.randn(30, 100)  # 30 筆資料, 100 個特徵
tensor_data = model(tensor_data)    # 前向傳播
print(tensor_data)
### 實作結果
# 只要 model(tensor_data) 能做一次前向傳播即可

### 練習 3：搭建手寫數字辨識模型

In [ ]:
#@title 3.1 搭建模型
### 情境：地方的郵局委託你做手寫數字辨識，請搭建一個三層的網路的模型
### 模型設計為(784, 512) -> (ReLU) -> (512, 10)，
### 參考：（4.1 搭建網路）、（2.3 激活函數）

class Three_layer_model(nn.Module):
    def __init__(self):
        super(Three_layer_model, self).__init__()

        ### 實作練習 ###

        self.flatten = nn.Flatten() # 資料攤平，2維度圖片，轉為1維資料

        self.layer_1 = ?
        self.relu = ?
        self.layer_2 = ?

        ### 實作練習 ###

    def forward(self, x):

        ### 實作練習 ###

        x = self.flatten(x)
        x = ?
        x = ?
        x = ?

        ### 實作練習 ###

        return x

image, label = test_dataset[3]
model = Three_layer_model()
data = model(image) # 測試
print(data.shape)

### 實作練習
# 模型建立完成，測試的結果可以顯示資料，則成功
# 即可進行下方的模型訓練

In [ ]:
#@title 3.2 訓練模型
# 用於記錄損失和準確率的列表
train_losses = []
train_accuracies = []

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

num_epochs = 5
# 訓練模型
for epoch in range(num_epochs):
    model.train()
    running_train_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in train_loader:
        # 前向傳播
        outputs = model(images)
        loss = criterion(outputs, labels)

        # 反向傳播和優化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    train_loss = running_train_loss / len(train_loader)
    train_accuracy = 100 * correct_train / total_train
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)

    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, ')

In [ ]:
#@title 3.3 評估模型
model.eval()
running_test_loss = 0.0
correct_test = 0
total_test = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        loss = criterion(outputs, labels)
        running_test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_test += labels.size(0)
        correct_test += (predicted == labels).sum().item()

test_loss = running_test_loss / len(test_loader)
test_accuracy = 100 * correct_test / total_test

print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

In [ ]:
#@title 3.4 觀看訓練過程
# 視覺化損失值
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss', color='blue')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss Over Epochs')
plt.legend()

# 視覺化準確率
plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Train Accuracy', color='blue')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Accuracy Over Epochs')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
#@title 3.5 觀看照片結果 -> [製作圖片](https://docs.google.com/drawings)
image_file = 'test.png'

if os.path.isfile(image_file):
  image = cv2.imread(image_file)
  image = cv2.resize(image, (28,28), interpolation=cv2.INTER_AREA)
  gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  gray_image = gray_image.reshape((1, 1, 28, 28)) # 2維 擴增為 4 維度
  gray_image = gray_image / 255
  gray_image = torch.Tensor(gray_image)
  plt.imshow(image)

else:
  print('No file is found')

In [ ]:
#@title 3.6 模型評估
if os.path.isfile(image_file):
  model.eval()

  with torch.no_grad():
      outputs = model(gray_image)
      prob, predicted = torch.max(outputs.data, 1)
      print(f'預測類別：{predicted.item()}')

## 7.手寫數字辨識 + CNN + GPU 訓練

In [ ]:
#@title 7.1 定義資料處方式與資料集

# 前置設定
transform = transforms.Compose([
    transforms.ToTensor()
])

# 加載 MNIST 數據集
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=256, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=256, shuffle=False)

In [ ]:
#@title 7.2 建立模型與優化器
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()

    def forward(self, x):

        x = self.relu(self.conv1(x))
        x = self.pool1(x)

        x = self.relu(self.conv2(x))
        x = self.pool2(x)

        x = x.view(-1, 64 * 7 * 7)

        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = CNN()
image, label = test_dataset[3]

print(model(image))

In [ ]:
#@title 7.3 創建模型與訓練策略
model = CNN().to('cuda') # 放入 GPU 計算

# 設置損失函數和優化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
#@title 7.4 訓練模型
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        images = images.to('cuda') # 放入 GPU 計算
        labels = labels.to('cuda') # 放入 GPU 計算
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')

In [ ]:
#@title 7.6 測試模型
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to('cuda') # 放入 GPU 計算
        labels = labels.to('cuda') # 放入 GPU 計算
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Test Accuracy: {100 * correct / total:.2f}%')

## 隨堂練習 B

### 練習 增加網路的深度

In [ ]:
### 情境：對於以下的卷積神經網路進行改良，請幫它疊加更深的一層卷積 +Maxpooling
### 新的一層 Conv2d(64, 128)
### 參考：（7.2 建立模型與優化器）

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0) # (28,28) -> (14,14)

        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0) # (14,14) -> (7,7)

         ## 實作練習 ##

        self.conv3 = nn.Conv2d(in_channels= ? , out_channels= ? , kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)   # (7,7) -> (3,3)

        self.fc1 = nn.Linear(128 * 3 * 3, 128)

         ## 實作練習 ##
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()

    def forward(self, x):

        x = self.relu(self.conv1(x))
        x = self.pool1(x)

        x = self.relu(self.conv2(x))
        x = self.pool2(x)

        ## 實作練習 ##

        # 可參考上方的第一層與第二層
        x = ?
        x = ?

        ## 實作練習 ##

        x = x.view(-1, 128 * 3 * 3) ## 實作練習 ##

        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = CNN()
image, label = test_dataset[3]
data = model(image) # 測試
print(data.shape)
### 實作練習

### 訓練 & 測試

In [ ]:
# 創建模型實例
model = CNN().to('cuda') # 放入 GPU 計算

# 設置損失函數和優化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 訓練模型
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        images = images.to('cuda') # 放入 GPU 計算
        labels = labels.to('cuda') # 放入 GPU 計算
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')

# 測試模型
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to('cuda') # 放入 GPU 計算
        labels = labels.to('cuda') # 放入 GPU 計算
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Test Accuracy: {100 * correct / total:.2f}%')